In [1]:
from ethpandaops_python.preprocessor import Preprocessor
from eip4844_blob_data.panel_charts import start_interactive_panel, filter_data_seq
from eip4844_blob_data.polars_preprocess import create_slot_inclusion_df, create_slot_gas_bidding_df
from holoviews import opts
import nest_asyncio
import polars as pl
import panel as pn

nest_asyncio.apply()
pn.extension("tabulator", template="material", sizing_mode="stretch_width")
pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

### Data Query

In [2]:
# labeled blobs - https://dune.com/queries/3521610
sequencers_l2: dict[list[str], list[str]] = {
    "sequencer_addresses": [
        # should be the "from" addresses, this is what hilldobby SQL query does.
        # should also be proper checksum, not lowercase
        "0xC1b634853Cb333D3aD8663715b08f41A3Aec47cc",
        "0x5050F69a9786F081509234F1a7F4684b5E5b76C9",
        "0x6887246668a3b87F54DeB3b94Ba47a6f63F32985",
        "0x000000633b68f5D8D3a86593ebB815b4663BCBe0",
        "0x415c8893D514F9BC5211d36eEDA4183226b84AA7",
        "0xa9268341831eFa4937537bc3e9EB36DbecE83C7e",
        "0xcF2898225ED05Be911D3709d9417e86E0b4Cfc8f",
        "0x0D3250c3D5FAcb74Ac15834096397a3Ef790ec99",
        "0xC70ae19B5FeAA5c19f576e621d2bad9771864fe2",
        "0xC94C243f8fb37223F3EB2f7961F7072602A51B8B"
    ],
    "sequencer_names": [
        "arbitrum",
        "base",
        "optimism",
        "taiko",
        "blast",
        "linea",
        "scroll",
        "zksync",
        "paradex",
        "metal"
    ],
}

num_days: int = 7

In [3]:
def get_data() -> dict[str, pl.DataFrame]:
    return Preprocessor(
        blob_producer=sequencers_l2,
        period=num_days,
        network="mainnet",  # mainnet
    ).cached_data

In [4]:
# # retrieve data from database and store in memory
cached_data = get_data()


sequencer_names_list: list[str] = sorted(sequencers_l2["sequencer_names"])

# prepare dataframes
slot_inclusion_df = create_slot_inclusion_df(
    cached_data).join(
    pl.from_dict(sequencers_l2),
    left_on="from",
    right_on="sequencer_addresses",
    how="left",
    coalesce=True
).select('slot', 'slot_time', 'hash', 'fill_percentage', 'submission_count', 'slot_inclusion_rate', 'sequencer_names', 'meta_network_name')

slot_gas_bidding_df = create_slot_gas_bidding_df(
    cached_data).select('block_number', 'hash', 'base_tx_fee_eth', 'priority_tx_fee_eth', "base_fee_per_gas",
                        "priority_fee_gas", 'total_tx_fee_eth', 'priority_fee_bid_percent_premium')

slot_inclusion_joined_df = slot_inclusion_df.join(
    slot_gas_bidding_df, on="hash", how="left", coalesce=True
)

2024-06-10 is within a day of 2024-06-09


In [5]:
slot_inclusion_joined_df.drop_nulls().sort(
    by='base_fee_per_gas', descending=True)

slot,slot_time,hash,fill_percentage,submission_count,slot_inclusion_rate,sequencer_names,meta_network_name,block_number,base_tx_fee_eth,priority_tx_fee_eth,base_fee_per_gas,priority_fee_gas,total_tx_fee_eth,priority_fee_bid_percent_premium
u32,datetime[ms],str,f64,u32,f64,str,str,u64,f64,f64,f64,f64,f64,f64
9246684,2024-06-07 18:17:11,"""0xd943e6b2c18c0f660bef9057ddff778034ff2a179c811ec1a059acaa464cc6fb""",99.35000000000001,1,40,"""base""","""mainnet""",20041685,0.0023859579910530003,0.00033600000000000004,113.617047193,16,0.002721957991053,0.123
9246684,2024-06-07 18:17:11,"""0xd943e6b2c18c0f660bef9057ddff778034ff2a179c811ec1a059acaa464cc6fb""",99.35000000000001,1,40,"""base""","""mainnet""",20041685,0.0023859579910530003,0.00033600000000000004,113.617047193,16,0.002721957991053,0.123
9246684,2024-06-07 18:17:11,"""0xd943e6b2c18c0f660bef9057ddff778034ff2a179c811ec1a059acaa464cc6fb""",99.35000000000001,1,40,"""base""","""mainnet""",20041685,0.0023859579910530003,0.00033600000000000004,113.617047193,16,0.002721957991053,0.123
9246684,2024-06-07 18:17:11,"""0xd943e6b2c18c0f660bef9057ddff778034ff2a179c811ec1a059acaa464cc6fb""",99.35000000000001,1,40,"""base""","""mainnet""",20041685,0.0023859579910530003,0.00033600000000000004,113.617047193,16,0.002721957991053,0.123
9246684,2024-06-07 18:17:11,"""0xd943e6b2c18c0f660bef9057ddff778034ff2a179c811ec1a059acaa464cc6fb""",99.35000000000001,1,40,"""base""","""mainnet""",20041685,0.0023859579910530003,0.00033600000000000004,113.617047193,16,0.002721957991053,0.123
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
9258854,2024-06-09 10:51:11,"""0x5c240f1ddff84fc4d600d0f3e5308e92d948cb9ad876325396ad8be7cd5e4131""",100,1,9,"""arbitrum""","""mainnet""",20053785,0.000497227625390877,0.000172797,2.8775246410000004,1,0.0006700246253908771,0.258
9258854,2024-06-09 10:51:11,"""0x5c240f1ddff84fc4d600d0f3e5308e92d948cb9ad876325396ad8be7cd5e4131""",100,1,9,"""arbitrum""","""mainnet""",20053785,0.000497227625390877,0.000172797,2.8775246410000004,1,0.0006700246253908771,0.258
9258854,2024-06-09 10:51:11,"""0xf4f9e3fe654de7806cf360c3562e132ab1a89fd7afac1845bd9678c620f72fcc""",7.12,1,3,"""taiko""","""mainnet""",20053785,0.000607221004793102,0.00021102200000000003,2.8775246410000004,1,0.0008182430047931021,0.258


In [6]:
slot_inclusion_joined_df.drop_nulls().unique().group_by('sequencer_names').agg(
    pl.col('base_tx_fee_eth').sum().alias('total_base_fees_eth'),
    pl.col('priority_tx_fee_eth').sum().alias('total_priority_fees_eth'),
    pl.col('total_tx_fee_eth').sum().alias('total_eth_fees'),
    pl.col('priority_fee_gas').mean().alias('avg_priority_gas_bid'),
).sort(by='sequencer_names')

sequencer_names,total_base_fees_eth,total_priority_fees_eth,total_eth_fees,avg_priority_gas_bid
str,f64,f64,f64,f64
"""arbitrum""",11.279320519100184,0.9040326159999996,12.18335313510018,1.0045558086560364
"""base""",1.6155599179761082,0.4772880000000022,2.092847917976109,3.8528564163417527
"""blast""",0.495698628462975,0.1469790000000001,0.6426776284629748,3.7487948580610606
"""linea""",5.5623157116435396,0.6113755293463411,6.173691240989879,1.3913711816075258
"""metal""",0.152806535727321,0.033641999999999984,0.18644853572732095,2.005006257822278
"""optimism""",0.763579147921065,0.1388100000000005,0.9023891479210651,2.288781163434903
"""paradex""",3.3758645145358863,0.028098964800000036,3.403963479335887,0.09999999999999978
"""scroll""",13.294968835797043,0.10918766425740803,13.404156500054444,0.09704550746164446
"""taiko""",88.73702521420965,36.428677011699875,125.16570222590958,4.981337035634652


### run the dashboard

In [7]:
# # retrieve data from database and store in memory
cached_data = get_data()

# prepare dataframes
slot_inclusion_df = create_slot_inclusion_df(
    cached_data).join(
    pl.from_dict(sequencers_l2),
    left_on="from",
    right_on="sequencer_addresses",
    how="left",
    coalesce=True
).select('slot', 'slot_time', 'hash', 'blob_hashes_length', 'fill_percentage', 'submission_count', 'slot_inclusion_rate', 'sequencer_names', 'meta_network_name')

slot_gas_bidding_df = create_slot_gas_bidding_df(
    cached_data).select('block_number', 'hash', 'base_tx_fee_eth', 'priority_tx_fee_eth', "base_fee_per_gas",
                        "priority_fee_gas", 'total_tx_fee_eth', 'priority_fee_bid_percent_premium')

slot_inclusion_joined_df = slot_inclusion_df.join(
    slot_gas_bidding_df, on="hash", how="left", coalesce=True
)

# contains multiple dataframes, packaged up for dashboard sorting
filtered_data_dict = filter_data_seq(
    sequencers_l2["sequencer_names"], slot_inclusion_joined_df, cached_data
)

sequencer_names_list: list[str] = sorted(sequencers_l2["sequencer_names"])

dashboard = start_interactive_panel(filtered_data_dict, sequencer_names_list)
dashboard.show()

2024-06-10 is within a day of 2024-06-09


Launching server at http://localhost:37917


Gtk-Message: 10:06:26.502: Failed to load module "canberra-gtk-module"
Gtk-Message: 10:06:26.520: Failed to load module "canberra-gtk-module"


Opening in existing browser session.
